<a href="https://colab.research.google.com/github/EduBernardo/mba-projects/blob/master/Analisando_dados_de_acidentes-Entrega_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Atividade Final Data Engineering - Big Data Analytics
# Nome: Eduardo Luiz Bernardo da Silva     R.A: 2200568
# Nome:     R.A:
# Nome:     R.A:
# Nome:     R.A:
# Nome:     R.A:

In [85]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [86]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"
 
# tornar o pyspark "importável"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').getOrCreate()

## Avaliação final

# Exercício 1

In [87]:
# carregando arquivos da tabela de Boletins

df_bol = spark.read.csv("si-bol-2019.csv", sep=';', header=True, encoding = 'latin1', inferSchema=True)

Tabela boletins contém os seguintes campos:
NUMERO_BOLETIM;
DATA_HORA_BOLETIM;
DATA_INCLUSAO;
TIPO_ACIDENTE;
DESC_TIPO_ACIDENTE;
COD_TEMPO;
DESC_TEMPO;
COD_PAVIMENTO;
PAVIMENTO;
COD_REGIONAL;
DESC_REGIONAL;
ORIGEM_BOLETIM;
LOCAL_SINALIZADO;
VELOCIDADE_PERMITIDA;
COORDENADA_X;
COORDENADA_Y;
HORA_INFORMADA;
INDICADOR_FATALIDADE;
VALOR_UPS;
DESCRIÇÃO_UPS;
DATA_ALTERACAO_SMSA;
VALOR_UPS_ANTIGA;
DESCRIÇÃO_UPS_ANTIGA;

In [88]:
# carregando arquivos da tabela de Localização

df_log = spark.read.csv("si-log-2019.csv", sep=';', header=True, encoding = 'latin1', inferSchema=True)

Tabela localização contém os seguintes campos:
Nº_boletim;
data_boletim;
 Nº_municipio;
nome_municipio;
seq_logradouros;
Nº_logradouro;
tipo_logradouro;
nome_logradouro;
tipo_logradouro_anterior;
nome_logradouro_anterior;
Nº_bairro;
nome_bairro;
tipo_bairro;
descricao_tipo_bairro;
Nº_imovel;
Nº_imovel_proximo;


In [89]:
# carregando arquivos da tabela de Envolvidos

df_env = spark.read.csv("si_env-2019.csv", sep=';', header=True, encoding = 'latin1', inferSchema=True)

Tabela envolvidos contém os seguintes campos:
num_boletim;
data_hora_boletim;
Nº_envolvido;
condutor;
cod_severidade;
desc_severidade;
sexo;
cinto_seguranca;
Embreagues;
Idade;
nascimento;
categoria_habilitacao;
descricao_habilitacao;
declaracao_obito;
cod_severidade_antiga;
especie_veiculo;
pedestre;
passageiro;

In [ ]:
# apresentando as tabelas
df_bol.show(25)
df_log.show(25)
df_env.show(25)

# Exercício 2

In [ ]:
#Join entre tabela boletins e localização
df_join_bol_log = df_bol.join(df_log, df_bol[' NUMERO_BOLETIM'] == df_log['Nº_boletim'], 'left')
df_join_bol_log.show(10)

In [ ]:
#Join entre tabela resultante do join anterior + tabela envolvidos
df_join_total = df_join_bol_log.join(df_env, df_bol[' NUMERO_BOLETIM'] == df_env['num_boletim'], 'left')
df_join_total.show(10)

In [ ]:
# Criando visão específica com os campos solicitados
import pyspark.sql.functions as F

df_base = df_join_total.groupby(' data_hora_boletim',' TIPO_ACIDENTE', ' DESC_TIPO_ACIDENTE',' DESC_TEMPO',' PAVIMENTO',' desc_severidade',' cinto_seguranca',
                      ' Embreagues',' Idade',' sexo','especie_veiculo','nome_municipio','nome_bairro','tipo_logradouro','nome_logradouro').agg(F.sum(' Nº_envolvido').alias('total_envolvidos'))
df_base.show(20,truncate=False)

# Exercício 3

In [ ]:
# Remover valores nulos
from pyspark.sql.functions import col,when,ltrim

df_base_limpo=df_base.select([when(col(c)=='',None).otherwise(col(c)).alias(c) for c in df_base.columns])

df_base_limpo = df_base.fillna('especie_veiculo',"Não se aplica")

# Removendo dados inválidos

df_base_limpo = df_base.withColumn('especie_veiculo',
                                 F.when(df_base['especie_veiculo'] == '', 'Não especificado'))

df_base_limpo = df_base.withColumn(' sexo',
                                 F.when(df_base[' sexo'] == 'F', 'Feminino') \
                                 .when(df_base[' sexo'] == 'M', 'Masculino') \
                                 .otherwise('Não especificado'))
df_base = df_base_limpo
df_base.show(400)

In [ ]:
# Validando o data type das colunas data_hora_boletim e especie_veiculo
print(df_base.schema[' data_hora_boletim'].dataType)
print(df_base.schema['especie_veiculo'].dataType)

# Exercício 4

In [ ]:
# Consulta 1 - Número de acidentes por mês/ano
from pyspark.sql.functions import *

df_base_mes_ano = df_base.withColumn('mes_ano', substring(' data_hora_boletim', 4,7))
df_acidentes_mes_ano = df_base_mes_ano.groupby('mes_ano').agg(F.count(' TIPO_ACIDENTE').alias('total_acidentes')).sort(col("mes_ano").asc())
df_acidentes_mes_ano.show(20,truncate=False)

In [ ]:
# Consulta 2 - Total de acidentes com vítima por bairro
df_acidentes_vitimas_bairro_filtrado = df_base.filter(F.col(' DESC_TIPO_ACIDENTE').contains(" COM VITIMA"))
df_acidentes_vitimas_bairro = df_acidentes_vitimas_bairro_filtrado.groupby('nome_bairro').agg(F.count(' TIPO_ACIDENTE').alias('total_acidentes'))
df_acidentes_vitimas_bairro.show(20,truncate=False)

In [ ]:
# Consulta 3 - Total de acidentes por tipo de pavimento e condição do tempo
df_acidentes_pavimento_tempo = df_base.groupby(' PAVIMENTO', ' DESC_TEMPO').agg(F.count(' TIPO_ACIDENTE').alias('total_acidentes'))
df_acidentes_pavimento_tempo.show(20,truncate=False)

In [ ]:
# Consulta 4 - Total de pessoas acidentadas por tipo de veiculo e pavimento
df_acidentes_veiculo_pavimento = df_base.groupby('especie_veiculo', ' PAVIMENTO').agg(F.count(' TIPO_ACIDENTE').alias('total_acidentes'))
df_acidentes_veiculo_pavimento.show(20,truncate=False)

# Exercício 5

Gráficos da Consulta 1




In [ ]:
# Convertendo em Pandas
pandas_df_acidentes_mes_ano = df_acidentes_mes_ano.toPandas()

In [ ]:
# Gráfico Pizza
pandas_df_acidentes_mes_ano.groupby('mes_ano').total_acidentes.sum().plot.pie(figsize=(8,8))

In [ ]:
# Gráfico Barras
pandas_df_acidentes_mes_ano.groupby(['mes_ano']).total_acidentes.sum().plot.barh(figsize=(8,8))

Gráficos da Consulta 3


In [115]:
# Convertendo em Pandas
pandas_df_acidentes_pavimento_tempo = df_acidentes_pavimento_tempo.toPandas()

In [ ]:
# Gráfico Barras
pandas_df_acidentes_pavimento_tempo.groupby([' PAVIMENTO', ' DESC_TEMPO']).total_acidentes.sum().unstack().plot.barh(figsize=(8,8))